<a href="https://colab.research.google.com/github/KEEHWANG/AI12_-_Section1_Project/blob/main/20220719_%ED%95%9C%EA%B5%AD%EC%96%B4_%EA%B0%9C%EC%B1%84%EB%AA%85_%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data

--2022-07-19 04:56:51--  https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data [following]
--2022-07-19 04:56:52--  https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16945023 (16M) [text/plain]
Saving to: ‘train_data’

train_data          100%[===================>]  16.16M  --.-KB/s    in 0.1s    

2022-07-19 04:56:52 (137 MB/s) - ‘train_data’ saved [16945023/16945

In [3]:
!pip install transformers
!pip install sentencepiece
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.7 MB/s 
     |████████████████████████████████| 101 kB 9.5 MB/s 
     |████████████████████████████████| 6.6 MB 4.7 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [4]:
train = pd.read_csv("train_data", names=['src', 'tar'], sep="\t")
train = train.reset_index()
train

,index,src,tar
0,1,비토리오,PER_B
1,2,양일,DAT_B
2,3,만에,-
3,4,영사관,ORG_B
4,5,감호,CVL_B
...,...,...,...
769060,2,어째,-
769061,3,뭔가,-
769062,4,수상쩍은,-
769063,5,좌담,-


In [5]:
train['src'] = train['src'].str.replace("．", ".", regex=False)

In [6]:
train.loc[train['src']=='.']

,index,src,tar
15,6,.,-
30,15,.,-
40,10,.,-
77,24,.,-
96,19,.,-
...,...,...,...
769013,11,.,-
769036,23,.,-
769053,17,.,-
769058,5,.,-


In [7]:
train['src'] = train['src'].astype(str)
train['tar'] = train['tar'].astype(str)

train['src'] = train['src'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', "", regex=True)

In [8]:
data = [list(x) for x in train[['index', 'src', 'tar']].to_numpy()]

In [9]:
print(data[:20])

[[1, '비토리오', 'PER_B'], [2, '양일', 'DAT_B'], [3, '만에', '-'], [4, '영사관', 'ORG_B'], [5, '감호', 'CVL_B'], [6, '용퇴', '-'], [7, '항룡', '-'], [8, '압력설', '-'], [9, '의심만', '-'], [10, '가율', '-'], [1, '이', '-'], [2, '음경동맥의', '-'], [3, '직경이', '-'], [4, '8', 'NUM_B'], [5, '19mm입니다', 'NUM_B'], [6, '.', '-'], [1, '9세이브로', 'NUM_B'], [2, '구완', '-'], [3, '30위인', 'NUM_B'], [4, 'LG', 'ORG_B']]


In [10]:
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [11]:
print(label_dict)

{'PER_B': 0, 'DAT_B': 1, '-': 2, 'ORG_B': 3, 'CVL_B': 4, 'NUM_B': 5, 'LOC_B': 6, 'EVT_B': 7, 'TRM_B': 8, 'TRM_I': 9, 'EVT_I': 10, 'PER_I': 11, 'CVL_I': 12, 'NUM_I': 13, 'TIM_B': 14, 'TIM_I': 15, 'ORG_I': 16, 'DAT_I': 17, 'ANM_B': 18, 'MAT_B': 19, 'MAT_I': 20, 'AFW_B': 21, 'FLD_B': 22, 'LOC_I': 23, 'AFW_I': 24, 'PLT_B': 25, 'FLD_I': 26, 'ANM_I': 27, 'PLT_I': 28, '[PAD]': 29}


In [12]:
print(index_to_ner)

{0: 'PER_B', 1: 'DAT_B', 2: '-', 3: 'ORG_B', 4: 'CVL_B', 5: 'NUM_B', 6: 'LOC_B', 7: 'EVT_B', 8: 'TRM_B', 9: 'TRM_I', 10: 'EVT_I', 11: 'PER_I', 12: 'CVL_I', 13: 'NUM_I', 14: 'TIM_B', 15: 'TIM_I', 16: 'ORG_I', 17: 'DAT_I', 18: 'ANM_B', 19: 'MAT_B', 20: 'MAT_I', 21: 'AFW_B', 22: 'FLD_B', 23: 'LOC_I', 24: 'AFW_I', 25: 'PLT_B', 26: 'FLD_I', 27: 'ANM_I', 28: 'PLT_I', 29: '[PAD]'}


In [13]:
tups = []
temp_tup = []
temp_tup.append(data[0][1:])
sentences = []
targets = []
for i, j, k in data:
  
  if i != 1:
    temp_tup.append([j,label_dict[k]])
  if i == 1:
    if len(temp_tup) != 0:
      tups.append(temp_tup)
      temp_tup = []
      temp_tup.append([j,label_dict[k]])

tups.pop(0)

[['비토리오', 'PER_B']]

In [14]:
print(tups[0], tups[1])

[['비토리오', 0], ['양일', 1], ['만에', 2], ['영사관', 3], ['감호', 4], ['용퇴', 2], ['항룡', 2], ['압력설', 2], ['의심만', 2], ['가율', 2]] [['이', 2], ['음경동맥의', 2], ['직경이', 2], ['8', 5], ['19mm입니다', 5], ['.', 2]]


In [15]:
sentences = []
targets = []
for tup in tups:
  sentence = []
  target = []
  sentence.append("[CLS]")
  target.append(label_dict['-'])
  for i, j in tup:
    sentence.append(i)
    target.append(j)
  sentence.append("[SEP]")
  target.append(label_dict['-'])
  sentences.append(sentence)
  targets.append(target)

In [16]:
sentences[0]

['[CLS]',
 '비토리오',
 '양일',
 '만에',
 '영사관',
 '감호',
 '용퇴',
 '항룡',
 '압력설',
 '의심만',
 '가율',
 '[SEP]']

In [17]:
targets[0]

[2, 0, 1, 2, 3, 4, 2, 2, 2, 2, 2, 2]

In [18]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [19]:
tokenizer.tokenize("대한민국 만세.")

['대한민국', '만', '##세', '.']

In [20]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):

    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    tokenized_sentence.extend(tokenized_word)
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [21]:
tokenized_texts_and_labels = [
                              tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(sentences, targets)]

In [22]:
print(tokenized_texts_and_labels[:2])
# [(문장, 개체들), (문장, 개체들),...] 형식으로 저장되어 있음.

[(['[CLS]', '비', '##토', '##리', '##오', '양', '##일', '만', '##에', '영', '##사', '##관', '감', '##호', '용', '##퇴', '항', '##룡', '압', '##력', '##설', '의', '##심', '##만', '가', '##율', '[SEP]'], [2, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), (['[CLS]', '이', '음', '##경', '##동', '##맥', '##의', '직', '##경', '##이', '8', '19', '##mm', '##입', '##니다', '.', '[SEP]'], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2])]


In [23]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [24]:
tokenized_texts[1]

['[CLS]',
 '이',
 '음',
 '##경',
 '##동',
 '##맥',
 '##의',
 '직',
 '##경',
 '##이',
 '8',
 '19',
 '##mm',
 '##입',
 '##니다',
 '.',
 '[SEP]']

In [25]:
labels[1]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2]

In [26]:
print(np.quantile(np.array([len(x) for x in tokenized_texts]), 0.975))
max_len = 88
bs = 32

96.0


In [27]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [28]:
input_ids[1]

array([   101,   9638,   9634,  31720,  18778, 118915,  10459,   9707,
        31720,  10739,    129,  10270,  17525,  58303,  48345,    119,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [29]:
tags = pad_sequences([lab for lab in labels], maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [70]:
tags = pad_sequences(labels, maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [76]:
label_dict["[PAD]"]

29

In [69]:
tags[0]

array([ 2,  0,  0,  0,  0,  1,  1,  2,  2,  3,  3,  3,  4,  4,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29])

In [31]:
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])

In [62]:
?tokenizer.convert_tokens_to_ids

In [68]:
# tokenizer.convert_tokens_to_ids("[SEP]")
tokenizer.convert_ids_to_tokens(20309)

'##호'

In [57]:
attention_masks[3]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [59]:
len(attention_masks[0])

88

In [60]:
input_ids[0]

array([   101,   9379,  26444,  12692,  28188,   9543,  18392,   9248,
        10530,   9574,  12945,  20595,   8848,  20309,   9603, 119362,
         9959, 118888,   9527,  28143,  31928,   9637,  71013,  19105,
         8843, 119183,    102,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [61]:
attention_masks[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [32]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)

In [33]:
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [34]:
# TPU 작동을 위해 실행
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.73.17.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.73.17.74:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [35]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
  
  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

In [36]:
def _compute_gradients(tensor, var_list):
    grads = tf.gradients(tensor, var_list)
    return [grad if grad is not None else tf.zeros_like(var)
    for var, grad in zip(var_list, grads)]

In [37]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  nr_model = create_model()
  nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 88)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 88)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 88,                                            

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3


1843/1843 [==============================] - 317s 126ms/step - loss: 0.2287 - sparse_categorical_accuracy: 0.9348 - val_loss: 0.1555 - val_sparse_categorical_accuracy: 0.9527
Epoch 2/3
1843/1843 [==============================] - 170s 92ms/step - loss: 0.1429 - sparse_categorical_accuracy: 0.9563 - val_loss: 0.1415 - val_sparse_categorical_accuracy: 0.9571
Epoch 3/3
1843/1843 [==============================] - 169s 92ms/step - loss: 0.1153 - sparse_categorical_accuracy: 0.9642 - val_loss: 0.1362 - val_sparse_categorical_accuracy: 0.9591


In [38]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [51]:
y_predicted = nr_model.predict([val_inputs, val_masks])

In [40]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()]
f_label.remove("[PAD]")

In [41]:
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

       PER_B       0.84      0.84      0.84     11508
       DAT_B       0.90      0.84      0.87      4354
           -       0.95      0.96      0.95    136806
       ORG_B       0.86      0.82      0.84     13511
       CVL_B       0.76      0.81      0.78     15837
       NUM_B       0.94      0.92      0.93     11161
       LOC_B       0.84      0.75      0.79      5912
       EVT_B       0.82      0.80      0.81      3887
       TRM_B       0.83      0.68      0.75      6052
       TRM_I       0.32      0.53      0.40       670
       EVT_I       0.75      0.83      0.79      1833
       PER_I       0.67      0.72      0.69      1578
       CVL_I       0.45      0.42      0.44       869
       NUM_I       0.71      0.76      0.74      1538
       TIM_B       0.85      0.90      0.87       630
       TIM_I       0.89      0.85      0.87       219
       ORG_I       0.61      0.71      0.66      1481
       DAT_I       0.82    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
def ner_inference(test_sentence):

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))

In [43]:
#금액 확인
ner_inference("금액은요 7만9천4백6십구원입니다. 1원이라도 더 들어가면 불량이 뜨니까 정확하게 입금해주셔야 해요.")

-	금
-	액
-	은
-	요
NUM_B	7
NUM_B	만
NUM_B	9
NUM_B	천
NUM_B	4
NUM_B	백
NUM_B	6
NUM_B	십
NUM_B	구
NUM_B	원
NUM_B	입
NUM_B	니다
NUM_B	.
NUM_B	1
NUM_B	원이
NUM_B	라
NUM_B	도
-	더
-	들어
-	가
-	면
-	불
-	량
-	이
-	뜨
-	니
-	까
-	정
-	확
-	하게
-	입
-	금
-	해
-	주
-	셔
-	야
-	해
-	요
-	.


In [44]:
# 날짜 확인
ner_inference("아 다름이 아니라일단 11월23일인가? 안내장이 날라왔어요.")

-	아
-	다
-	름이
DAT_B	아니라
DAT_B	일
DAT_B	단
DAT_B	11월
DAT_B	23
DAT_B	일
DAT_B	인
DAT_B	가
-	?
-	안
-	내
-	장이
-	날
-	라
-	왔
-	어
-	요
-	.


In [45]:
#계좌번호 확인
ner_inference("가상계좌를 불러드릴게요 6984738393 한번 불러보실래요? 6984738393")

AFW_B	가
AFW_B	상
AFW_B	계
AFW_B	좌
AFW_B	를
-	불
-	러
-	드
-	릴
-	게
-	요
NUM_B	698
NUM_B	47
NUM_B	38
NUM_B	3
NUM_B	9
NUM_B	3
NUM_B	한
-	번
-	불
-	러
-	보
-	실
-	래
-	요
-	?
NUM_B	698
NUM_B	47
NUM_B	38
NUM_B	3
NUM_B	9
NUM_B	3


In [46]:
#금액 확인(모두 한글인 경우)
ner_inference("금액은요 칠만구천사백육십구원입니다. 일원이라도 더 들어가면 불량이 뜨니까 정확하게 입금해주셔야 해요.")

-	금
-	액
-	은
-	요
ORG_B	칠
ORG_B	만
ORG_B	구
ORG_B	천
ORG_B	사
ORG_B	백
ORG_B	육
ORG_B	십
ORG_B	구
ORG_B	원
ORG_B	입
ORG_B	니다
-	.
-	일
-	원이
-	라
-	도
-	더
-	들어
-	가
-	면
-	불
-	량
-	이
-	뜨
-	니
-	까
-	정
-	확
-	하게
-	입
-	금
-	해
-	주
-	셔
-	야
-	해
-	요
-	.


In [50]:
# 금액 확인(모두 문자인 경우)
ner_inference("이근늠이요 지금 통장에 백팔십만원 입금이 됐거든요?")

-	[UNK]
-	지
-	금
-	통
-	장에
NUM_B	백
NUM_B	팔
NUM_B	십
NUM_B	만
NUM_B	원
-	입
-	금
-	이
-	됐
-	거
-	든
-	요
-	?


In [49]:
# 날짜 확인(모두 문자인 경우)
ner_inference("아 다름이 아니라일단 십일월이십삼일인가? 안내장이 날라왔어요.")

-	아
-	다
DAT_B	름이
DAT_B	아니라
-	일
-	단
DAT_B	십
DAT_B	일
DAT_B	월
DAT_B	이
DAT_B	십
DAT_B	삼
DAT_B	일
DAT_B	인
DAT_B	가
-	?
-	안
-	내
-	장이
-	날
-	라
-	왔
-	어
-	요
-	.
